In [ ]:
pip install mlflow xgboost

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# dagshub setup
import dagshub
dagshub.init(repo_owner='valiant.shabri', repo_name='dagster', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=be8776e2-ccbe-45ca-915e-8e84a20089f5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ab30a5f856bc73291ee72980e1da59e380a171ed70b4ce55fe682565bf5974a8




Output()

Accessing as valiant.shabri

Initialized MLflow to track repo "valiant.shabri/dagster"

Repository valiant.shabri/dagster initialized!

In [11]:
# import os
# os.environ['MLFLOW_TRACKING_USERNAME'] = 'valiant.shabri'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd37b33ad4e0564f52162d90248e477d373a699f1'
# os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/valiant.shabri/dagster.mlflow'

# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("https://dagshub.com/valiant.shabri/dagster.mlflow")
# mlflow.set_tracking_uri("http://localhost:5000")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/08/21 14:10:41 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 14:10:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/valiant.shabri/dagster.mlflow/#/experiments/0/runs/331fff88e2ee47adaedfa7b06d7fa159.
2024/08/21 14:10:42 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/valiant.shabri/dagster.mlflow/#/experiments/0.
2024/08/21 14:10:53 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 14:10:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/valiant.shabri/dagster.mlflow/#/experiments/0/runs/75e8918146d34b1f85d81781049201b8.
2024/08/21 14:10:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https:/

In [12]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 147eedbf35ed4c789e4c57ea349f868e


Successfully registered model 'XGB-Smote'.
2024/08/22 08:40:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.
2024/08/22 08:40:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/valiant.shabri/dagster.mlflow/#/experiments/0/runs/147eedbf35ed4c789e4c57ea349f868e.
2024/08/22 08:40:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/valiant.shabri/dagster.mlflow/#/experiments/0.


In [14]:
model_version = 2
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [15]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1724291023681, current_stage='None', description='', last_updated_timestamp=1724291023681, name='anomaly-detection-prod', run_id='147eedbf35ed4c789e4c57ea349f868e', run_link='', source='models:/XGB-Smote/2', status='READY', status_message='', tags={}, user_id='', version='1'>

In [16]:
# model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])